# Tensor

Tensor（テンソル）は配列や行列に非常によく似た特殊なデータ構造です。
PyTorchでは、モデルの入力と出力、およびモデルのパラメータをエンコードするためにテンソルを使用します。

テンソルは[NumPy](https://numpy.org/)のndarrayに似ていますが、テンソルはGPUや他のハードウェアアクセラレータ上で処理することができる点が異なります。一方で、テンソルとNumPyの配列は同じメモリを共有することができ、データをコピーする必要がありません。
さらにテンソルは、自動微分のために最適化されています（これについては後ほどAutogradのセクションで詳しく説明します）。

In [5]:
import torch
import numpy as np

## Tensorの初期化


テンソルは様々な方法で初期化することができます。

**データから直接作成する**

テンソルはデータから直接作成することができます。このとき、データ型は自動的に推論されます。



In [6]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

**NumPy array から生成する**

テンソルはNumPyの配列から作成することができます（逆も同様です）。


In [7]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

**他のテンソルから生成する**

新しいテンソルは、明示的に上書きされない限り、引数のテンソルの特性(形状、データ型)を保持します。



In [8]:
x_ones = torch.ones_like(x_data) # x_dataの特性(形状、データ型)を保持します
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # x_dataのデータ型を上書きします
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.2047, 0.1791],
        [0.4233, 0.2341]]) 



**乱数や定数値を生成する**

``shape`` はテンソルの次元を表すタプルです。  
``shape``を使ってテンソルの次元を設定して乱数や定数を生成してみます。



In [9]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.1634, 0.7509, 0.8228],
        [0.5666, 0.2860, 0.9644]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


## テンソルの属性

テンソルは属性（attribute）として、次の3つを保持しています。
- 形状：`shape`
- データ型：`dtype`
- 保持されているデバイス：`device`

In [10]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## テンソルの演算

算術、線形代数、行列操作（転置、インデックス、スライス）、サンプリングなど、100以上のテンソル演算が可能です。  
公式のドキュメントは[こちら](https://pytorch.org/docs/stable/torch.html)です。

これらの操作はそれぞれGPUで実行できます(当然ですが、CPUよりも高速に実行できます)。  
Colabを使っている場合、`ランタイム→ランタイムのタイプを変更→ハードウェアアクセラレータ`でGPUを割り当てます。

デフォルトでは、テンソルは CPU 上で作成されます。そのため、テンソルを明示的に GPU に移動させる必要があります。
``.to`` メソッドを用いると GPU に移動することができます。  
※大きいテンソルをCPU→GPUあるいはGPU→CPUへと転送させると時間とメモリの両面でボトルネックになる可能性があります。


In [11]:
# この記述でGPUが使える場合はテンソルをGPUへ転送することができます
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

NumPy APIIと操作が似ているので、NumPy APIに慣れていればTensor APIは簡単に使うことができます。

**NumPyライクな配列・スライスの操作をする**



In [12]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


**テンソルを結合する**

``torch.cat`` を使うと、一連のテンソルを指定した次元に沿って連結することができます。
[torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html)も参照してください。  
※``torch.cat`` とは微妙に異なる結合方法もあります。


In [13]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


**算術演算子を使う**

2つのテンソル間の行列の掛け算を計算します。y1, y2, y3 は同じ値になります。  
`tensor.T` はテンソルの転置を返します。
@は行列のための演算子です(参考：[Qiitaの記事](https://qiita.com/sci_Haru/items/aa0b35fb0fb71cabc808))。



In [14]:
y1 = tensor @ tensor.T
print(f"y1: {y1}")
y2 = tensor.matmul(tensor.T)
print(f"y2: {y2}")
y3 = torch.rand_like(y1)
print(f"y3: {y3}")

torch.matmul(tensor, tensor.T, out=y3)

y1: tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])
y2: tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])
y3: tensor([[0.3484, 0.6255, 0.2917, 0.9612],
        [0.2573, 0.7901, 0.9924, 0.1935],
        [0.2392, 0.4113, 0.9303, 0.4336],
        [0.9871, 0.4340, 0.6279, 0.5244]])


tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])

これは要素ごとの積を計算します。z1, z2, z3 は同じ値になります。


In [15]:
z1 = tensor * tensor
print(f"z1: {z1}")
z2 = tensor.mul(tensor)
print(f"z2: {z2}")
z3 = torch.rand_like(tensor)
print(f"z3: {z3}")

torch.mul(tensor, tensor, out=z3)

z1: tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
z2: tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
z3: tensor([[0.7557, 0.3293, 0.0132, 0.2565],
        [0.9940, 0.3879, 0.0986, 0.4425],
        [0.2598, 0.8808, 0.2925, 0.9191],
        [0.5158, 0.3605, 0.7257, 0.7024]])


tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

**シングルエレメントテンソル、単元テンソルを使う**  
要素が一つのテンソルは`item()`メソッドを使うことでPythonの数値に変換することができます。  
例）合計値を算出する`sum()`の結果を格納した`agg`の場合



In [16]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


**インプレース演算を使う**

結果を元の変数（＝オペランド）に格納する演算をインプレース演算と呼びます。  
これらは接尾辞 ``_`` で表現されます。
例えば、 ``x.copy_(y)`` や ``x.t_()`` は ``x``(=
オペランド)
 を変更することになります。

In [17]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


インプレース演算はメモリを節約できますが、自動微分のアルゴリズムに問題が生じる可能性があるので使用はお勧めしません。



## NumPyとのブリッジ
CPU上のテンソルとNumPyの配列は、メモリを共有することができます。
一方を変更するともう一方も変更されます。

### TensorからNumPyへの変換



In [18]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


テンソルの変更はNumPyの配列に反映される。

In [19]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


### NumPyからTensorへの変換



In [20]:
n = np.ones(5)
t = torch.from_numpy(n)

NumPy配列の変更もテンソルに反映される。



In [21]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]


--------------


